In [ ]:
from libs.env import *

from datetime import datetime as dt


from libs.agents import Agent, AgentConfig, create_agent, invoke_agent
from dawgpyl.libs.base import Directories, Timestamp
from libs.graphs import run_team_workflow
from libs.io import read_text, write_text
# from libs.apis import APIS

# from rich import print as richprint
from IPython.display import Markdown as md
from flask.views import F

dirs = Directories()
log_prefix = "log_"
filename = log_prefix + Timestamp().date + ".joblib"
log_file_path = os.path.join(dirs.logs, filename)

# print_heading("Available APIs and Models",'green')
# eprint(MODELS)
# print("\n")

print_heading("Available Agent Personas", "green")
eprint([x for x in list(AGENTS.keys())])
print("\n")

print_heading("Project Directories", "green")
print_dict(dirs.__dict__, "green")

In [2]:
save_log = True

project_type = "small"

project_goal = """

We would like to eat Korean food in downtown Boston, MA. 
Can you provide an introduction to Korean cuisine and some advice for Korean charcoal restaurants? 


"""

project = Project(project_type,project_goal)


In [ ]:
eprint(project)

In [ ]:
project.teams[0].members[0]

In [3]:
team = project.teams[0]

In [ ]:
eprint(team.project.teams[0].project.teams[0].project.teams[0].project.teams[0])

In [ ]:
team_graph = StateGraph(team)

In [ ]:


with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    team_graph = StateGraph(team)

    # Add entry and exit nodes
    team_graph.set_entry_point(team.config.graph_config["entry"])
    if AGENTS[team.config.graph_config["finish"]]["needs_review"]:
        team_graph.set_finish_point(f"{team.config.graph_config['finish']}_reviewer")
    else:
        team_graph.set_finish_point(team.config.graph_config["finish"])

    # Add each agent on the team as a node
    for member in team.members:
        team_graph.add_node(
            member.name,
            member.invoke,
        )

    # Get pre-defined workflow order from config
    edge_order = team.config.graph_config["edge_order"]

    for edge_idx, member_name in enumerate(edge_order):
        if edge_idx < len(edge_order) - 1:
            next_member = edge_order[edge_idx + 1]
        else:
            next_member = END

        # Define the runnable that is used to see if an agent is finished
        @chain
        def check_member_finished(member_name=member_name):
            if member_name in team.members_finished:
                return str(True)
            else:
                return str(False)

        if AGENTS[member_name]["needs_review"]:
            team_graph.add_conditional_edges(
                member_name,
                check_member_finished,
                {"True": next_member, "False": f"{member_name}_reviewer"},
            )
            if (
                f"{member_name}_reviewer"
                != f'{team.config.graph_config["finish"]}_reviewer'
            ):
                team_graph.add_edge(f"{member_name}_reviewer", next_member)
            else:
                pass
        else:
            team_graph.add_edge(member_name, next_member)

team_graph.compile()

# check_member_finished.get_graph()

In [ ]:
final_state = run_team_workflow(project_type, project_goal)

In [ ]:
# TODO: Make a helper function that finds the index of a given agent's name
# NOTE: I might already have this... something like find_agent('responder')???

agent_name = "responder"

### Display Response
try:
    print_heading(agent_name, "green")
    display(
        md(final_state.outputs.last[agent_name]["message"]["solution"]["description"])
    )
except:
    for member in final_state.fetch_member_names():
        print_heading(member)
        eprint(final_state.outputs.last[member]["message"])
        # print(response.choices[0].message.content)

In [ ]:
### Save Response
if save_response:
    log_response(response, chat_config, dirs)


if save_log:
    filename = Timestamp().filestamp
    filepath = os.path.join(dirs.logs, filename)
    model_artifact = {
        "run_datetime": Timestamp().iso,
        "model_config": "default",
        "user_prompt": project_goal,
        "final_response": final_state.final_answers,
    }
    log_text = f"log = [{model_artifact}]"
    write_file(log_text, filepath)


# if save_log:
#         log = read_file(log_file_path)
#         log = eval(log.replace("log = ", ""))
#         model_artifact = {
#             "run_datetime": Timestamp().iso,
#             "model_config": 'default',
#             "user_prompt": project_goal,
#             "final_response": final_state.final_answers,
#         }
#         log.append(model_artifact)
#         log_text = f"log = {log}"
#         write_file(log_text, log_file_path)

In [ ]:
# eprint(final_state.members[3].outputs.last_message)


### Save Response
if save_response:
    log_response(response, chat_config, dirs)